##### Experiment, sheet 11:
Tot nu toe zie ik 3 manieren om tekst+tekstcoordinaten te implementeren:
1. op basis van een onveranderlijke lijst van basissegmenten (zoals de lines in Republic), 
2. op basis van een initiele segmentatie die wordt bepaald door hoe de te verwerken bron al is opgedeeld (zoals het TEI voorbeeld, met head, section, paragraph en page), waarbij de segmentatie later is te verfijnen, 
3. op basis van een initiele segmentatie, zoals in geval 2, waarbij fijnmaziger segmentatie kan worden toegevoegd op basis van relatieve char offset tov het initiele segment in kwestie. 

Ik heb cases 1 en 2 inmiddels geimplementeerd als subclasses van een abstract class SegmentedText. Case 2 werkt (per definitie) al, want identiek aan sheet 10.

In deze sheet pas ik de recentste Republic code uit sheet 7 aan, om gebruik te maken van IndexedSegmentedText, in plaats van van een simpele list. IndexedSegmentedText doet overigens niet veel meer dan een wrapper voor een list zijn.

Mogelijk vervolg:
- zoek uit of ik ook een abstracte Anchor class nodig heb
- instantieer voor beide cases de juiste SegmentedText en gebruik die in de REST services

In [1]:
import sys
sys.path.append('../un-t-ann-gle')

from extractors import json_extractor

# We want to load 'text containers' that contain more or less contiguous text and are as long as practically
# possible. Container size is determined by pragmatic reasons, e.g. technical (performance) or user driven
# (corresponding with all scans in a book or volume). This functions returns all component files IN TEXT ORDER.
# Examples: sorted list of files, part of IIIF manifest.
import glob


def expand_glob_pattern(pattern:str)->list:
    session_file_names = (f for f in glob.glob(pattern))
    return sorted(session_file_names)

glob_pattern = "../data/sessions/meeting-1705*"
files = expand_glob_pattern(glob_pattern)
(all_textlines,all_annotations) = json_extractor.process(files)

In [2]:
all_annotations[-20:]

[{'label': 'lines',
  'image_coords': {'left': 2591,
   'right': 3313,
   'top': 2226,
   'bottom': 2302,
   'height': 76,
   'width': 722},
  'begin_anchor': 21854,
  'id': 'NL-HaNA_1.01.02_3760_0032-page-63-col-0-tr-3-line-2',
  'end_anchor': 21854},
 {'label': 'lines',
  'image_coords': {'left': 2586,
   'right': 3416,
   'top': 2274,
   'bottom': 2351,
   'height': 77,
   'width': 830},
  'begin_anchor': 21855,
  'id': 'NL-HaNA_1.01.02_3760_0032-page-63-col-0-tr-3-line-3',
  'end_anchor': 21855},
 {'label': 'lines',
  'image_coords': {'left': 2636,
   'right': 3247,
   'top': 2324,
   'bottom': 2396,
   'height': 72,
   'width': 611},
  'begin_anchor': 21856,
  'id': 'NL-HaNA_1.01.02_3760_0032-page-63-col-0-tr-3-line-4',
  'end_anchor': 21856},
 {'label': 'lines',
  'image_coords': {'left': 2590,
   'right': 3417,
   'top': 2371,
   'bottom': 2449,
   'height': 78,
   'width': 827},
  'begin_anchor': 21857,
  'id': 'NL-HaNA_1.01.02_3760_0032-page-63-col-0-tr-3-line-5',
  'end_ancho

Lees resolutie info in uit de resultaten van queries naar Marijn's resolutie-index, voor de betreffende 7 zittingsdagen. Vooralsnog neem ik alleen resolutie id en line-ids van de eerste en laatste regel van de resolutie mee.

In [3]:
import json

resolution_annotations=[]

# def get_resolution_files_for_container(text_container):
#     path = "../data/resolutions/*-resolutions.json"
#     resolution_file_names = (f for f in glob.glob(path))
#     return sorted(resolution_file_names)

def res_traverse(node, line_ids):
    # find the list that represents the children, each child is a dict, assume first list is the correct one
    children = []
    label_of_children = ''
    
    # assume, first list in dict are the children
    for key,val in node.items():
        # HACK ALERT! assumption that first list contains children has exceptions
        if type(val) == list and key != 'paragraphs' and key != 'evidence':
            children = val
            label_of_children = key
            break 
    
    if len(children) == 0:        # if no children, do your 'leaf node thing'
        line_ids.append(node['metadata']['id'])
    else:                         # if non-leaf node, first visit children     
        for child in children:
            res_traverse(child,line_ids)
                            
    return

# In case of presence of a hierarchical structure, processing/traversal typically starts from a root element.

def get_res_root_element(file):
    with open(file, 'r') as myfile:
        resolution_file=myfile.read() 
        
    resolution_data = json.loads(resolution_file)      
    return resolution_data['hits']['hits']

path = "../data/resolutions/*-resolutions.json"
# for f_name in get_resolution_files_for_container('volume-1705-1'):    
for f_name in expand_glob_pattern(path):    
    # get list of resolution 'hits'
    hits = get_res_root_element(f_name)
    print(f_name)
    for hit in hits:
        # each hit corresponds with a resolution
        resolution_line_ids = []
        res_traverse(hit['_source'],resolution_line_ids)
        
        resolution_info = {'label' : 'resolutions','begin_anchor' : resolution_line_ids[0], \
                                      'end_anchor': resolution_line_ids[len(resolution_line_ids)-1], 'id': hit['_id']}
        resolution_annotations.append(resolution_info)

resolution_annotations

../data/resolutions/1705-01-02-resolutions.json
../data/resolutions/1705-01-03-resolutions.json
../data/resolutions/1705-01-06-resolutions.json
../data/resolutions/1705-01-07-resolutions.json
../data/resolutions/1705-01-09-resolutions.json
../data/resolutions/1705-01-10-resolutions.json
../data/resolutions/1705-01-11-resolutions.json


[{'label': 'resolutions',
  'begin_anchor': 'NL-HaNA_1.01.02_3760_0011-page-20-column-0-tr-0-line-26',
  'end_anchor': 'NL-HaNA_1.01.02_3760_0012-page-22-column-0-tr-0-line-7',
  'id': 'meeting-1705-01-02-session-1-resolution-27'},
 {'label': 'resolutions',
  'begin_anchor': 'NL-HaNA_1.01.02_3760_0012-page-22-column-0-tr-0-line-8',
  'end_anchor': 'NL-HaNA_1.01.02_3760_0012-page-22-column-0-tr-0-line-20',
  'id': 'meeting-1705-01-02-session-1-resolution-28'},
 {'label': 'resolutions',
  'begin_anchor': 'NL-HaNA_1.01.02_3760_0012-page-22-column-0-tr-0-line-21',
  'end_anchor': 'NL-HaNA_1.01.02_3760_0012-page-22-column-0-tr-0-line-43',
  'id': 'meeting-1705-01-02-session-1-resolution-29'},
 {'label': 'resolutions',
  'begin_anchor': 'NL-HaNA_1.01.02_3760_0012-page-22-column-0-tr-0-line-44',
  'end_anchor': 'NL-HaNA_1.01.02_3760_0012-page-22-column-1-tr-0-line-2',
  'id': 'meeting-1705-01-02-session-1-resolution-30'},
 {'label': 'resolutions',
  'begin_anchor': 'NL-HaNA_1.01.02_3760_0012-

Maak voor alle line-ids een dict aan van line-id versus text_index

In [4]:
line_ids_vs_indexes = {}
for line in all_annotations:
    if line['label'] == 'lines':
        line_ids_vs_indexes.update({line['id'] : line['begin_anchor']})
        
line_ids_vs_indexes

{'NL-HaNA_1.01.02_3760_0008-page-15-col-0-tr-0-line-8': 47076,
 'NL-HaNA_1.01.02_3760_0008-page-15-col-0-tr-0-line-9': 47077,
 'NL-HaNA_1.01.02_3760_0008-page-15-col-0-tr-0-line-10': 47078,
 'NL-HaNA_1.01.02_3760_0008-page-15-col-0-tr-0-line-11': 47079,
 'NL-HaNA_1.01.02_3760_0008-page-15-col-0-tr-0-line-12': 47080,
 'NL-HaNA_1.01.02_3760_0008-page-15-col-0-tr-0-line-13': 47081,
 'NL-HaNA_1.01.02_3760_0008-page-15-col-0-tr-0-line-14': 47082,
 'NL-HaNA_1.01.02_3760_0008-page-15-col-0-tr-0-line-15': 47083,
 'NL-HaNA_1.01.02_3760_0008-page-15-col-0-tr-4-line-2': 47084,
 'NL-HaNA_1.01.02_3760_0008-page-15-col-0-tr-4-line-3': 47085,
 'NL-HaNA_1.01.02_3760_0008-page-15-col-0-tr-1-line-0': 47086,
 'NL-HaNA_1.01.02_3760_0008-page-15-col-0-tr-1-line-1': 47087,
 'NL-HaNA_1.01.02_3760_0008-page-15-col-0-tr-1-line-2': 47088,
 'NL-HaNA_1.01.02_3760_0008-page-15-col-0-tr-1-line-3': 47089,
 'NL-HaNA_1.01.02_3760_0008-page-15-col-0-tr-1-line-4': 47090,
 'NL-HaNA_1.01.02_3760_0008-page-15-col-0-tr-1-li

Voor alle resoluties, voeg begin_index en end_index toe.
Opmerking: marijn gebruikt bij zittingsdagen 'col' in de ids, en bij resoluties 'column'. Fix mbv regex.

In [5]:
for res in resolution_annotations:
    num_errors = 0
    try:
        res['begin_anchor'] = re.sub(r'-column-', r'-col-', res['begin_anchor'])
        res['end_anchor'] = re.sub(r'-column-', r'-col-', res['end_anchor'])
    
        res['begin_anchor'] = line_ids_vs_indexes[res['begin_anchor']]
        res['end_anchor'] = line_ids_vs_indexes[res['end_anchor']]
    except:
        res['begin_anchor'] = 0
        res['end_anchor'] = 0
        num_errors += 1
    if num_errors > 0:
        print(f"number of lookup errors: {num_errors}")
    
resolution_annotations

number of lookup errors: 1
number of lookup errors: 1
number of lookup errors: 1
number of lookup errors: 1
number of lookup errors: 1
number of lookup errors: 1
number of lookup errors: 1
number of lookup errors: 1
number of lookup errors: 1
number of lookup errors: 1
number of lookup errors: 1
number of lookup errors: 1
number of lookup errors: 1
number of lookup errors: 1
number of lookup errors: 1
number of lookup errors: 1
number of lookup errors: 1
number of lookup errors: 1
number of lookup errors: 1
number of lookup errors: 1
number of lookup errors: 1
number of lookup errors: 1
number of lookup errors: 1
number of lookup errors: 1
number of lookup errors: 1
number of lookup errors: 1
number of lookup errors: 1
number of lookup errors: 1
number of lookup errors: 1
number of lookup errors: 1
number of lookup errors: 1
number of lookup errors: 1
number of lookup errors: 1
number of lookup errors: 1
number of lookup errors: 1
number of lookup errors: 1
number of lookup errors: 1
n

[{'label': 'resolutions',
  'begin_anchor': 0,
  'end_anchor': 0,
  'id': 'meeting-1705-01-02-session-1-resolution-27'},
 {'label': 'resolutions',
  'begin_anchor': 0,
  'end_anchor': 0,
  'id': 'meeting-1705-01-02-session-1-resolution-28'},
 {'label': 'resolutions',
  'begin_anchor': 0,
  'end_anchor': 0,
  'id': 'meeting-1705-01-02-session-1-resolution-29'},
 {'label': 'resolutions',
  'begin_anchor': 0,
  'end_anchor': 0,
  'id': 'meeting-1705-01-02-session-1-resolution-30'},
 {'label': 'resolutions',
  'begin_anchor': 0,
  'end_anchor': 0,
  'id': 'meeting-1705-01-02-session-1-resolution-31'},
 {'label': 'resolutions',
  'begin_anchor': 0,
  'end_anchor': 0,
  'id': 'meeting-1705-01-02-session-1-resolution-32'},
 {'label': 'resolutions',
  'begin_anchor': 0,
  'end_anchor': 0,
  'id': 'meeting-1705-01-02-session-1-resolution-33'},
 {'label': 'resolutions',
  'begin_anchor': 0,
  'end_anchor': 0,
  'id': 'meeting-1705-01-02-session-1-resolution-34'},
 {'label': 'resolutions',
  'beg

In [6]:
all_annotations.extend(resolution_annotations)

In [7]:
from annotation import asearch
asearch.get_annotations_at_anchor(2675, all_annotations)

[]

Zoek alle annotaties die overlappen met een interval. Dat interval kan ontleent worden aan begin_index en end_index van een resolutie.

In [8]:
for a in asearch.get_annotations_of_type('resolutions', all_annotations):
    print(a)

{'label': 'resolutions', 'begin_anchor': 0, 'end_anchor': 0, 'id': 'meeting-1705-01-02-session-1-resolution-27'}
{'label': 'resolutions', 'begin_anchor': 0, 'end_anchor': 0, 'id': 'meeting-1705-01-02-session-1-resolution-28'}
{'label': 'resolutions', 'begin_anchor': 0, 'end_anchor': 0, 'id': 'meeting-1705-01-02-session-1-resolution-29'}
{'label': 'resolutions', 'begin_anchor': 0, 'end_anchor': 0, 'id': 'meeting-1705-01-02-session-1-resolution-30'}
{'label': 'resolutions', 'begin_anchor': 0, 'end_anchor': 0, 'id': 'meeting-1705-01-02-session-1-resolution-31'}
{'label': 'resolutions', 'begin_anchor': 0, 'end_anchor': 0, 'id': 'meeting-1705-01-02-session-1-resolution-32'}
{'label': 'resolutions', 'begin_anchor': 0, 'end_anchor': 0, 'id': 'meeting-1705-01-02-session-1-resolution-33'}
{'label': 'resolutions', 'begin_anchor': 0, 'end_anchor': 0, 'id': 'meeting-1705-01-02-session-1-resolution-34'}
{'label': 'resolutions', 'begin_anchor': 0, 'end_anchor': 0, 'id': 'meeting-1705-01-02-session-1

In [9]:
for a in asearch.get_annotations_overlapping_with(2813,2825,all_annotations):
    print(a)

Combineer beide: alle annotaties van een bepaald type, binnen een interval

In [10]:
for a in asearch.get_annotations_of_type_overlapping('resolutions',2599,2850,all_annotations):
    print(a)

In [11]:
from textservice import segmentedtext

def get_textlines_between(begin,end,annotations): 
    textlines = segmentedtext.IndexedSegmentedText()
    for line_annot in asearch.get_annotations_of_type_overlapping('lines',begin,end,annotations):
        textlines.append((line_annot['begin_anchor'],all_textlines[line_annot['begin_anchor']]))
    return textlines
        

In [12]:
# get text for (text segment of) resolution
for a in get_textlines_between(2813,2825,all_annotations):
    print(a)

TypeError: 'IndexedSegmentedText' object is not iterable

Vraag nu de tekst op voor de kolom waar deze resolutie deel van uitmaakt

In [ ]:
# resolutiegrenzen 2813 en 2825 waren eerder al opgevraagd middels 'get_annotations_of_type'
# vraag eerst columns op, die overlappen met die grenzen
begin = 2813
end = 2825
column_annots = asearch.get_annotations_of_type_overlapping('columns',begin,end,all_annotations)

# voor ieder column, print column_id, print column text
for col_annot in column_annots:
    print(f"column id: {col_annot['id']}")
    print(f"{col_annot['begin_anchor']}   {col_annot['end_anchor']}\n")
    
    for a in get_textlines_between(col_annot['begin_anchor'],col_annot['end_anchor'],all_annotations):
        print(a)

Volgende stap: definieren van named entities. Voorlopig alleen door te verwijzen naar de index van de line(s) waar deze entities op voorkomen

In [ ]:
entity_segments = [(2814,2814,'per','Arnoldus Boomhouwer'),(2832,2833,'per','van Rabenpeé'),\
                   (2819,2819,'loc','Roermond')]

id_suffix = 1
entity_annotations = []
for es in entity_segments:
    ent_annot = {'label':'entities','begin_anchor':es[0],'end_anchor':es[1],\
                 'id':'NL-HaNA_1.01.02_3760_entity-'+str(id_suffix),\
                 'entity_type': es[2],'entity_text':es[3]}
    entity_annotations.append(ent_annot)
    id_suffix += 1

entity_annotations

In [ ]:
all_annotations.extend(entity_annotations)

In [ ]:
for a in asearch.get_annotations_of_type('entities', all_annotations):
    print(a)

In [ ]:
# get text for (text segment of) entity
for a in get_textlines_between(2832,2833,all_annotations):
    print(a)

In [ ]:
# get text for (text segment of) entity
for a in get_textlines_between(2819,2819,all_annotations):
    print(a)

In [ ]:
# get all entities for a specific session

# first get session that contains our example resolution from 2813 to 2825
for a in asearch.get_annotations_of_type_overlapping('sessions',2599,2850,all_annotations):
    print(a)

In [ ]:
# then, get overlapping entities
for a in asearch.get_annotations_of_type_overlapping('entities',2568,3093,all_annotations):
    print(a)

Experiment: voeg een 'image_range' element toe voor iedere resolutie

In [ ]:
resolution_annotations

In [ ]:
# voor iedere resolutie, bepaal image_range en voeg deze toe

def get_bounding_box_for(annotations): 
    ann_list = list(annotations) # because a generator can only be used once
    
    min_left = min([ann['image_coords']['left'] for ann in ann_list if 'image_coords' in ann])
    max_right = max([ann['image_coords']['right'] for ann in ann_list if 'image_coords' in ann])
    min_top = min([ann['image_coords']['top'] for ann in ann_list if 'image_coords' in ann])
    max_bottom = max([ann['image_coords']['bottom'] for ann in ann_list if 'image_coords' in ann])
    height = max_bottom - min_top
    width = max_right - min_left

    return {'left': min_left, 'right': max_right, 'top': min_top, 'bottom': max_bottom, 'height': height, 'width': width}

def add_image_range(ann):
    ann['image_range'] = []
    
    ann_begin=ann['begin_anchor']
    ann_end=ann['end_anchor']
        
    # loop over scans die overlappen met de annotatie
    for a in asearch.get_annotations_of_type_overlapping('scanpage',ann_begin,ann_end,all_annotations):
        bounding_boxes = []
        image_url = a['iiif_url']
        
        scan_begin=a['begin_anchor']
        scan_end=a['end_anchor']
        
        # loop over alle kolommen op de betreffende scan. Per kolom, bereken bounding box voor 
        # overlappende resolutieregels
        for clm in asearch.get_annotations_of_type_overlapping('columns',scan_begin,scan_end,all_annotations):
            clm_begin=clm['begin_anchor']
            clm_end=clm['end_anchor']
            
            # bepaal overlap_begin en overlap_end indexes voor kolom
            overlap_begin=max(ann_begin, clm_begin)
            overlap_end=min(ann_end, clm_end)
                        
            # bepaal hieruit de bounding box coords voor deze kolom
            if overlap_end-overlap_begin >= 0: # resolution and column are overlapping
                bounding_box=get_bounding_box_for(asearch.get_annotations_of_type_overlapping('lines',\
                                                        overlap_begin,overlap_end,all_annotations))
                bounding_boxes.append(bounding_box)
        
        ann['image_range'].append((image_url, bounding_boxes))
        print(ann['image_range'])
    return
    
for r in resolution_annotations:
    add_image_range(r)                

Tussenconclusie: het is mogelijk voor een resolutie alle omsluitende rectangles af te leiden, zelfs als de resolutietekst over meerdere kolommen en/of meerdere scans doorloopt. In principe zijn uit deze image_ranges IIIF image urls af te leiden voor de resolutieonderdelen op de scans, of de hele scans zijn op te halen, en de resolutiedelen daarop zijn te omkaderen.

Omdat het leuk is: genereer nog even een list van IIIF image urls per resolutie (en zet ze dan in je browser even naast elkaar.

In [ ]:
resolution_annotations

In [ ]:
def add_region_links(ann):
    region_links = []
    try:
        for image_url, regions in ann['image_range']:
            for coords in regions:
                # construct iiif_url from image_url and coords
                coord_str = f"{coords['left']},{coords['top']},{coords['width']},{coords['height']}"
                region_url = re.sub(r'(full)/(,\d+)', rf'{coord_str}/\1', image_url)
                region_links.append(region_url)
    except:
        print('error: annotation without image range')
        
    ann['region_links'] = region_links
    return
    
for res in resolution_annotations:
    add_region_links(res)

In [ ]:
resolution_annotations

In [ ]:
for a in asearch.get_annotations_of_type('resolutions', all_annotations):
    print(a['region_links'])

In [ ]:
asearch.get_annotation_by_id('meeting-1705-01-02-session-1-resolution-27', all_annotations)

In [ ]:
for a in asearch.get_annotations_of_type_overlapping('resolutions',130,130,all_annotations):
    print(a['region_links'])

In [ ]:
for a in get_textlines_between(127,147,all_annotations):
    print(a)

In [ ]:
for a in asearch.get_annotations_of_type('resolutions', all_annotations):
    print(json.dumps(a, sort_keys=False, indent=2))

In [ ]:
all_textlines.slice(20,30)

In [ ]:
# write all_textlines to a file
with open(datadir+'all_textlines.json', 'w') as filehandle:
    json.dump(all_textlines, filehandle, cls=segmentedtext.SegmentEncoder)

Add image_range and region_links to the entity_annotations

In [ ]:
entity_annotations

In [ ]:
for ea in entity_annotations:
    add_image_range(ea)

entity_annotations

In [ ]:
for ea in entity_annotations:
    add_region_links(ea)
    
entity_annotations

In [ ]:
for a in asearch.get_annotations_of_type('entities', all_annotations):
    print(json.dumps(a, sort_keys=False, indent=2))

In [ ]:
for t in all_textlines.slice(1148,1164):
    print(t)

In [ ]:
for a in asearch.get_annotations_of_type('resolutions', all_annotations):
    if len(a['region_links']) > 1:
        print(a['id'])

In [ ]:
asearch.get_annotation_by_id('meeting-1705-01-02-session-1-resolution-27', all_annotations)

In [ ]:
# write all_annotations to a file
with open(datadir+'republic_annotations.json', 'w') as filehandle:
    json.dump(all_annotations, filehandle, cls=segmentedtext.AnchorEncoder)